In [114]:
import pandas as pd
import numpy as np

1. Загрузим и выведем в консоль датафрейм

In [115]:
df = pd.read_csv("/content/drive/MyDrive/data/Uber.csv")
df

,Date,Time,State,PuFrom,Address,Street
0,2014.07.06,14:30,NY ...,MANHATTAN,50,MURRAY ST
1,2014.07.04,7:15,NY ...,MANHATTAN,143,AVENUE B
2,2014.07.05,5:45,NY ...,MANHATTAN,125,CHRISTOPHER ST
3,2014.07.06,4:30,NY ...,MANHATTAN,217,E 7 ST
4,2014.07.05,7:45,NY ...,MANHATTAN,521,W 26 ST
...,...,...,...,...,...,...
194987,2014.09.28,21:50,JFK DELTA TERM 4 P/U AREA C ...,Jfk,,
194988,2014.09.28,7:00,NY ...,MANHATTAN,112,W 78 ST
194989,2014.09.28,15:30,NY ...,MANHATTAN,601,W 57 ST
194990,2014.09.28,19:55,JFK DELTA TERM 2 CURBSIDE ...,NaN,,


На всякий проверили, что NaN это не строка, а отсутствие значения (float)

In [116]:
val = df.loc[194990, 'PuFrom']
print(type(val))

<class 'float'>


Посмотрим количество отсутствующих значений (пропусков) по полям

In [117]:
df.isna().sum()

,0
Date,0
Time,0
State,0
PuFrom,37030
Address,0
Street,0


Познакомимся с датафреймом с помощью info:

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194992 entries, 0 to 194991
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Date     194992 non-null  object
 1   Time     194992 non-null  object
 2   State    194992 non-null  object
 3   PuFrom   157962 non-null  object
 4   Address  194992 non-null  object
 5   Street   194992 non-null  object
dtypes: object(6)
memory usage: 8.9+ MB


describe:

In [119]:
df.describe()

,Date,Time,State,PuFrom,Address,Street
count,194992,194992,194992,157962,194992,194992
unique,92,1440,1001,1264,4328,9566
top,2014.09.18,15:00,NY ...,MANHATTAN,,
freq,2795,3674,133408,113625,54397,54148


1) Очистим данные от строк, в которых есть NaN или пустые значения. Алгоритм: сначала все пустые строки (табуляция, пробел, отсутствие символа) преобразуем в NaN, потом удалим все NaN. Для оптимзации все строки пустые мы обозначим через регулярку, чтобы сразу предусмотреть все варианты

In [120]:
df_cleaned = df.replace(r'^\s*$', np.nan, regex=True).dropna()
df_cleaned = df.dropna()
df_cleaned

,Date,Time,State,PuFrom,Address,Street
0,2014.07.06,14:30,NY ...,MANHATTAN,50,MURRAY ST
1,2014.07.04,7:15,NY ...,MANHATTAN,143,AVENUE B
2,2014.07.05,5:45,NY ...,MANHATTAN,125,CHRISTOPHER ST
3,2014.07.06,4:30,NY ...,MANHATTAN,217,E 7 ST
4,2014.07.05,7:45,NY ...,MANHATTAN,521,W 26 ST
...,...,...,...,...,...,...
194986,2014.09.29,6:00,NJ ...,RIDGEWOOD,756,BELMONT RD
194987,2014.09.28,21:50,JFK DELTA TERM 4 P/U AREA C ...,Jfk,,
194988,2014.09.28,7:00,NY ...,MANHATTAN,112,W 78 ST
194989,2014.09.28,15:30,NY ...,MANHATTAN,601,W 57 ST


2) Приведем данные столбца PuFrom к нижнему регистру. Сохраним изменения в новом датафрейме.

In [121]:
df_lower = df_cleaned.assign(PuFrom=df_cleaned['PuFrom'].str.lower())
df_lower

,Date,Time,State,PuFrom,Address,Street
0,2014.07.06,14:30,NY ...,manhattan,50,MURRAY ST
1,2014.07.04,7:15,NY ...,manhattan,143,AVENUE B
2,2014.07.05,5:45,NY ...,manhattan,125,CHRISTOPHER ST
3,2014.07.06,4:30,NY ...,manhattan,217,E 7 ST
4,2014.07.05,7:45,NY ...,manhattan,521,W 26 ST
...,...,...,...,...,...,...
194986,2014.09.29,6:00,NJ ...,ridgewood,756,BELMONT RD
194987,2014.09.28,21:50,JFK DELTA TERM 4 P/U AREA C ...,jfk,,
194988,2014.09.28,7:00,NY ...,manhattan,112,W 78 ST
194989,2014.09.28,15:30,NY ...,manhattan,601,W 57 ST


3) Преобразуем столбец Time в виде дроби. Поэтапно: применим к значеням Time как строкам функцию split() по разделителю ':', но для удобства с использованием атрибута expand=Trie (разводит части строки - времени по разным столбцам, а не в виде списка). Вычисляем числовые значения частей времени (минут и секунд), при этом числовые секунды делим на 60 (думаю, здесь не нужно пояснять почему 60). Складываем части между собой и сохраняем как строку, чтобы убрать лишние нули в конце (собственно, условий, что над временем будем производить математические операции не было). В новой копии датафрейма присваиваем вычисленные данные столбцу Time.

In [122]:
fraction = df_lower['Time'].str.split(':', expand=True)
min = fraction[0].astype(int)
sec =((fraction[1]).astype(int) / 60).astype(float)
res = (min + sec).astype(str)
df_fraction = df_lower.assign(Time = res)
df_fraction

,Date,Time,State,PuFrom,Address,Street
0,2014.07.06,14.5,NY ...,manhattan,50,MURRAY ST
1,2014.07.04,7.25,NY ...,manhattan,143,AVENUE B
2,2014.07.05,5.75,NY ...,manhattan,125,CHRISTOPHER ST
3,2014.07.06,4.5,NY ...,manhattan,217,E 7 ST
4,2014.07.05,7.75,NY ...,manhattan,521,W 26 ST
...,...,...,...,...,...,...
194986,2014.09.29,6.0,NJ ...,ridgewood,756,BELMONT RD
194987,2014.09.28,21.833333333333332,JFK DELTA TERM 4 P/U AREA C ...,jfk,,
194988,2014.09.28,7.0,NY ...,manhattan,112,W 78 ST
194989,2014.09.28,15.5,NY ...,manhattan,601,W 57 ST


Дополнительно на основе исходного, не обработанного датафрейма проделаем еще ряд операций: посмотрим, есть ли дубликаты строк: аргументу keep зададим False, чтобы вывести на экран дубликаты (включая первую строку) и отсортируем их, чтобы визуально увидеть их рядом и убедиться, что они реально есть. Хотя в задании не указана необходимость проверки дубликатов, думаю - это надо сделать.

In [123]:
duplicates = df[df.duplicated(keep=False)]
duplicates_sorted = duplicates.sort_values(by=df.columns.tolist()).reset_index(drop=True)
duplicates_sorted.head(100)
#df.head(100)

,Date,Time,State,PuFrom,Address,Street
0,2014.07.01,10:30,NY ...,MANHATTAN,1535,BROADWAY
1,2014.07.01,10:30,NY ...,MANHATTAN,1535,BROADWAY
2,2014.07.01,10:30,NY ...,MANHATTAN,485,5 AVE
3,2014.07.01,10:30,NY ...,MANHATTAN,485,5 AVE
4,2014.07.01,11:30,NY ...,BROOKLYN,70,IMLAY ST
...,...,...,...,...,...,...
95,2014.07.03,0:45,LGA DELTA MIDDLE ISLAND ...,NaN,,
96,2014.07.03,10:15,NY ...,MANHATTAN,151,W 54 ST
97,2014.07.03,10:15,NY ...,MANHATTAN,151,W 54 ST
98,2014.07.03,10:30,NY ...,MANHATTAN,15,W 72 ST


In [125]:
duplicates_sorted[duplicates_sorted.index==1]

,Date,Time,State,PuFrom,Address,Street
1,2014.07.01,10:30,NY ...,MANHATTAN,1535,BROADWAY


Теперь Удалим дубликаты, сохранив первую строку из них. И проверим, все ли норм удалилось

In [126]:
df_dedup = duplicates_sorted.drop_duplicates(keep='first')
duplicates_new = df_dedup[df_dedup.duplicated(keep=False)]
duplicates_new_sorted = duplicates_new.sort_values(by=df.columns.tolist()).reset_index(drop=True)
duplicates_new_sorted

,Date,Time,State,PuFrom,Address,Street


Да. дубликаты исчезли, при этом код ниже показывает, что первая строка дубликатов сохранилась

In [127]:
df_dedup[df_dedup.index==0]

,Date,Time,State,PuFrom,Address,Street
0,2014.07.01,10:30,NY ...,MANHATTAN,1535,BROADWAY


Теперь уже в датафрейме без дубликатов удалим строчки с пустыми значениями или NaN. Возможно, я неверно выполню задание, но я противник удалять строки и столбцы, если в них есть какие-то полезные значения. Например, в строках от 177383 и далее столбцы PuFrom, Address, Street принимают значения NaN, но при этом дата время и штат заказа указаны. Это можно объчснить, верятно тем, что не все заказы водители берут по адресу и сделанные из приложения. Возможно, эти пассажиры голосовали "от бордюра" и факт заказа есть, штат есть, дата и время зафиксировано самим водителем, а начальная точка неопределена и высадка пассажира была произведена по сигналу: "вот тут высади". Удалять эти заказы не правильно. Так как начальная и конечная точки строковые значения логично NaN заменить Unknown, а Address - числовое значение на 0. А если вдруг и значение штата NaN: можно заенить на USA. Что и предлагаю сделать.       

 Сначала все путые значения преобразуем в NaN и посчитаем сколько их в каждом столбце, чтобы понять, что делать. Так как по вырезке дата фрейма на экране не все видно.

In [129]:
cols_to_clean = ['State', 'PuFrom', 'Street']
df_changed = df_dedup[cols_to_clean].replace(r'^\s*$', pd.NA, regex=True)
df_res = df_changed[cols_to_clean].fillna('Unknown')
for col in cols_to_clean:
    nan_count = df_res[col].isna().sum()
    print(f"{col}: {nan_count}")
df_res.head(100)

State: 0
PuFrom: 0
Street: 0


,State,PuFrom,Street
0,NY ...,MANHATTAN,BROADWAY
2,NY ...,MANHATTAN,5 AVE
4,NY ...,BROOKLYN,IMLAY ST
6,EWR UNITED PICK-UP AREA 3 ...,Unknown,Unknown
8,JFK DELTA TERM 4 P/U AREA C ...,Unknown,Unknown
...,...,...,...
194,NY ...,Harrison,Tower Road
196,NY ...,MANHATTAN,W 54 ST
198,NY ...,SAYVILLE,RIVER RD
200,NY ...,MANHATTAN,W 54 ST
